# Coordinate & Angle Exploration

This notebook explores the spatial data in our gameplay recordings to determine:
1. What coordinate system Elden Ring uses
2. Whether angle data is accurate (verified against frames)
3. How to best preprocess coordinates for the model

**State attributes related to position/angle:**
- `[6]` HeroGlobalPosX
- `[7]` HeroGlobalPosY  
- `[8]` HeroGlobalPosZ
- `[9]` HeroAngle
- `[14]` NpcGlobalPosX
- `[15]` NpcGlobalPosY
- `[16]` NpcGlobalPosZ
- `[17]` NpcGlobalPosAngle


In [ ]:
import zarr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from pathlib import Path

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['figure.dpi'] = 100


In [ ]:
# Load dataset
dataset_path = Path('../../dataset/margit_100_256x144.zarr')
zarr_root = zarr.open(str(dataset_path), mode='r')

state_attrs = zarr_root.attrs['attributes']
print("State attributes:")
for i, attr in enumerate(state_attrs):
    print(f"  [{i:2d}] {attr}")


## 1. Load Episode Data


In [ ]:
# Select an episode
episode_idx = 1
episode = zarr_root[f'episode_{episode_idx}']

# DON'T load all frames - keep as zarr array for lazy loading
frames_zarr = episode['frames']  # Lazy - only loads when indexed
state = episode['state'][:]      # State is small, load it all
num_frames = frames_zarr.shape[0]

# Extract position and angle columns
hero_x = state[:, 6]   # HeroGlobalPosX
hero_y = state[:, 7]   # HeroGlobalPosY (vertical in most game engines)
hero_z = state[:, 8]   # HeroGlobalPosZ
hero_angle = state[:, 9]  # HeroAngle (radians, presumably)

npc_x = state[:, 14]   # NpcGlobalPosX
npc_y = state[:, 15]   # NpcGlobalPosY
npc_z = state[:, 16]   # NpcGlobalPosZ
npc_angle = state[:, 17]  # NpcGlobalPosAngle

print(f"Episode {episode_idx}: {num_frames} frames")
print(f"\nHero position range:")
print(f"  X: [{hero_x.min():.2f}, {hero_x.max():.2f}]")
print(f"  Y: [{hero_y.min():.2f}, {hero_y.max():.2f}]")
print(f"  Z: [{hero_z.min():.2f}, {hero_z.max():.2f}]")
print(f"  Angle: [{hero_angle.min():.2f}, {hero_angle.max():.2f}]")
print(f"\nNPC position range:")
print(f"  X: [{npc_x.min():.2f}, {npc_x.max():.2f}]")
print(f"  Y: [{npc_y.min():.2f}, {npc_y.max():.2f}]")
print(f"  Z: [{npc_z.min():.2f}, {npc_z.max():.2f}]")
print(f"  Angle: [{npc_angle.min():.2f}, {npc_angle.max():.2f}]")


## 2. Bird's Eye View: Hero vs NPC Movement

Plot the XZ plane (horizontal movement) to see how hero and boss move around the arena.


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Create animation on XY plane (rotated 180 degrees)
fig, ax = plt.subplots(figsize=(10, 12))

# Plot full path as faint background (negated for 180 degree rotation)
ax.plot(-hero_y, -hero_x, 'b-', alpha=0.15, linewidth=1)
ax.plot(-npc_y, -npc_x, 'r-', alpha=0.15, linewidth=1)

# Initialize animated elements
hero_trail, = ax.plot([], [], 'b-', linewidth=2, alpha=0.7, label='Hero')
npc_trail, = ax.plot([], [], 'r-', linewidth=2, alpha=0.7, label='NPC')
hero_dot, = ax.plot([], [], 'bo', markersize=12)
npc_dot, = ax.plot([], [], 'ro', markersize=12)
frame_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, fontsize=12,
                     verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

# Set axis limits with padding (negated for 180 degree rotation)
pad = 3
ax.set_xlim(-max(hero_y.max(), npc_y.max()) - pad, -min(hero_y.min(), npc_y.min()) + pad)
ax.set_ylim(-max(hero_x.max(), npc_x.max()) - pad, -min(hero_x.min(), npc_x.min()) + pad)
ax.set_xlabel('Y')
ax.set_ylabel('X')
ax.set_title('XY Plane Animation (180° rotated)')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

# Trail length (how many frames to show behind current position)
trail_len = 50

def init():
    hero_trail.set_data([], [])
    npc_trail.set_data([], [])
    hero_dot.set_data([], [])
    npc_dot.set_data([], [])
    frame_text.set_text('')
    return hero_trail, npc_trail, hero_dot, npc_dot, frame_text

def animate(i):
    # Show trail (negated for 180 degree rotation)
    start = max(0, i - trail_len)
    hero_trail.set_data(-hero_y[start:i+1], -hero_x[start:i+1])
    npc_trail.set_data(-npc_y[start:i+1], -npc_x[start:i+1])
    
    # Current position
    hero_dot.set_data([-hero_y[i]], [-hero_x[i]])
    npc_dot.set_data([-npc_y[i]], [-npc_x[i]])
    
    # Distance
    dist = np.sqrt((hero_x[i] - npc_x[i])**2 + (hero_y[i] - npc_y[i])**2)
    frame_text.set_text(f'Frame: {i}/{num_frames}\nDistance: {dist:.1f}')
    
    return hero_trail, npc_trail, hero_dot, npc_dot, frame_text

# Skip frames for speed (every 3rd frame)
frames = range(0, num_frames, 3)
anim = FuncAnimation(fig, animate, init_func=init, frames=frames, 
                     interval=50, blit=True)

plt.close()  # Don't show static figure
HTML(anim.to_jshtml())


In [ ]:
# Animation with game frame + minimap side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left: Game frame
ax_frame = axes[0]
ax_frame.set_title('Game Frame')
ax_frame.axis('off')

# Right: Minimap (XY plane, 180° rotated)
ax_map = axes[1]
ax_map.plot(-hero_y, -hero_x, 'b-', alpha=0.15, linewidth=1)
ax_map.plot(-npc_y, -npc_x, 'r-', alpha=0.15, linewidth=1)

# Initialize elements
frame_img = ax_frame.imshow(np.zeros((144, 256, 3), dtype=np.uint8))
hero_trail, = ax_map.plot([], [], 'b-', linewidth=2, alpha=0.7, label='Hero')
npc_trail, = ax_map.plot([], [], 'r-', linewidth=2, alpha=0.7, label='NPC')
hero_dot, = ax_map.plot([], [], 'bo', markersize=15)
npc_dot, = ax_map.plot([], [], 'ro', markersize=15)
line_between, = ax_map.plot([], [], 'g--', linewidth=2, alpha=0.5)
info_text = ax_map.text(0.02, 0.98, '', transform=ax_map.transAxes, fontsize=11,
                        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

# Set map limits
pad = 3
ax_map.set_xlim(-max(hero_y.max(), npc_y.max()) - pad, -min(hero_y.min(), npc_y.min()) + pad)
ax_map.set_ylim(-max(hero_x.max(), npc_x.max()) - pad, -min(hero_x.min(), npc_x.min()) + pad)
ax_map.set_xlabel('Y')
ax_map.set_ylabel('X')
ax_map.set_title('Minimap (XY Plane)')
ax_map.legend(loc='upper right')
ax_map.grid(True, alpha=0.3)
ax_map.set_aspect('equal')

trail_len = 30

def init():
    frame_img.set_array(np.zeros((144, 256, 3), dtype=np.uint8))
    hero_trail.set_data([], [])
    npc_trail.set_data([], [])
    hero_dot.set_data([], [])
    npc_dot.set_data([], [])
    line_between.set_data([], [])
    info_text.set_text('')
    return frame_img, hero_trail, npc_trail, hero_dot, npc_dot, line_between, info_text

def animate(i):
    # Load and display frame
    frame = frames_zarr[i]
    frame_rgb = np.transpose(frame, (1, 2, 0))[:, :, ::-1]  # CHW->HWC, BGR->RGB
    frame_img.set_array(frame_rgb)
    ax_frame.set_title(f'Frame {i}/{num_frames}')
    
    # Update minimap trails
    start = max(0, i - trail_len)
    hero_trail.set_data(-hero_y[start:i+1], -hero_x[start:i+1])
    npc_trail.set_data(-npc_y[start:i+1], -npc_x[start:i+1])
    
    # Current positions
    hero_dot.set_data([-hero_y[i]], [-hero_x[i]])
    npc_dot.set_data([-npc_y[i]], [-npc_x[i]])
    
    # Line between hero and NPC
    line_between.set_data([-hero_y[i], -npc_y[i]], [-hero_x[i], -npc_x[i]])
    
    # Info
    dist = np.sqrt((hero_x[i] - npc_x[i])**2 + (hero_y[i] - npc_y[i])**2)
    height = hero_z[i] - npc_z[i]
    info_text.set_text(f'Distance: {dist:.1f}\nHeight diff: {height:.2f}')
    
    return frame_img, hero_trail, npc_trail, hero_dot, npc_dot, line_between, info_text

# Every 5th frame for speed
frame_indices = range(0, num_frames, 5)
anim = FuncAnimation(fig, animate, init_func=init, frames=frame_indices,
                     interval=100, blit=True)

plt.tight_layout()
plt.close()
HTML(anim.to_jshtml())


In [ ]:
# Y offset correction - 8 bits / byte offset in NPC position data
Y_OFFSET = -8  # Confirmed offset

# Corrected NPC Y
npc_y_corrected = npc_y + Y_OFFSET

# Animation with corrected coordinates
fig = plt.figure(figsize=(14, 8))
ax_frame = fig.add_subplot(1, 2, 1)
ax_map = fig.add_subplot(1, 2, 2)

ax_frame.axis('off')

# Plot paths
ax_map.plot(-hero_y, -hero_x, 'b-', alpha=0.15, linewidth=1, label='Hero')
ax_map.plot(-npc_y_corrected, -npc_x, 'r-', alpha=0.15, linewidth=1, label=f'NPC (offset={Y_OFFSET})')
ax_map.set_xlabel('-Y')
ax_map.set_ylabel('-X')
ax_map.set_title(f'XY Plane with NPC Y offset = {Y_OFFSET}')
ax_map.legend()
ax_map.set_aspect('equal')
ax_map.grid(True, alpha=0.3)

frame_img = ax_frame.imshow(np.zeros((144, 256, 3), dtype=np.uint8))
hero_dot, = ax_map.plot([], [], 'bo', markersize=15)
npc_dot, = ax_map.plot([], [], 'ro', markersize=15)
line_between, = ax_map.plot([], [], 'g--', linewidth=2, alpha=0.5)

def init():
    frame_img.set_array(np.zeros((144, 256, 3), dtype=np.uint8))
    hero_dot.set_data([], [])
    npc_dot.set_data([], [])
    line_between.set_data([], [])
    return frame_img, hero_dot, npc_dot, line_between

def animate(i):
    frame = frames_zarr[i]
    frame_rgb = np.transpose(frame, (1, 2, 0))[:, :, ::-1]
    frame_img.set_array(frame_rgb)
    ax_frame.set_title(f'Frame {i}/{num_frames}')
    
    hero_dot.set_data([-hero_y[i]], [-hero_x[i]])
    npc_dot.set_data([-npc_y_corrected[i]], [-npc_x[i]])
    line_between.set_data([-hero_y[i], -npc_y_corrected[i]], [-hero_x[i], -npc_x[i]])
    
    return frame_img, hero_dot, npc_dot, line_between

frame_indices = range(0, num_frames, 5)
anim = FuncAnimation(fig, animate, init_func=init, frames=frame_indices,
                     interval=100, blit=True)

plt.tight_layout()
plt.close()
HTML(anim.to_jshtml())


In [ ]:
# Check the X coordinate comparison
print("X coordinate comparison:")
print(f"  Hero X range: [{hero_x.min():.2f}, {hero_x.max():.2f}]")
print(f"  NPC X range:  [{npc_x.min():.2f}, {npc_x.max():.2f}]")
print(f"\n  Hero X mean: {hero_x.mean():.2f}")
print(f"  NPC X mean:  {npc_x.mean():.2f}")
print(f"  Offset (NPC - Hero): {(npc_x.mean() - hero_x.mean()):.2f}")

# Plot X over time
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(hero_x, 'b-', label='Hero X', alpha=0.7)
ax.plot(npc_x, 'r-', label='NPC X', alpha=0.7)
ax.set_xlabel('Frame')
ax.set_ylabel('X coordinate')
ax.set_title('X coordinates over time')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

# Find frames where X coordinates are almost equal
x_diff = np.abs(hero_x - npc_x)
threshold = 1.0  # Adjust this - how close is "almost equal"

close_frames = np.where(x_diff < threshold)[0]
print(f"\nFrames where |hero_x - npc_x| < {threshold}:")
print(f"  Found {len(close_frames)} frames")

if len(close_frames) > 0:
    print(f"\n  First 20 instances:")
    for i, frame_idx in enumerate(close_frames[:20]):
        diff = hero_x[frame_idx] - npc_x[frame_idx]
        print(f"    Frame {frame_idx:4d}: hero_x={hero_x[frame_idx]:7.2f}, npc_x={npc_x[frame_idx]:7.2f}, diff={diff:+.4f}")

# Plot the difference over time
fig, ax = plt.subplots(figsize=(14, 3))
ax.plot(x_diff, 'g-', alpha=0.7)
ax.axhline(y=threshold, color='r', linestyle='--', label=f'threshold={threshold}')
ax.scatter(close_frames, x_diff[close_frames], c='red', s=20, zorder=5, label='Close frames')
ax.set_xlabel('Frame')
ax.set_ylabel('|Hero X - NPC X|')
ax.set_title('X coordinate difference over time')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()


In [ ]:
# For frames where X is almost equal, show Y difference
print("Frames where X is close - checking Y difference:")
print("-" * 80)

y_diffs_at_close_x = []
for frame_idx in close_frames[:30]:
    x_diff_val = hero_x[frame_idx] - npc_x[frame_idx]
    y_diff_val = hero_y[frame_idx] - npc_y[frame_idx]
    y_diffs_at_close_x.append(y_diff_val)
    print(f"Frame {frame_idx:4d}: hero_y={hero_y[frame_idx]:7.2f}, npc_y={npc_y[frame_idx]:7.2f}, y_diff={y_diff_val:+7.2f}")

y_diffs_at_close_x = np.array(y_diffs_at_close_x)
print("-" * 80)
print(f"\nY difference stats when X is aligned:")
print(f"  Mean:   {y_diffs_at_close_x.mean():+.2f}")
print(f"  Std:    {y_diffs_at_close_x.std():.2f}")
print(f"  Min:    {y_diffs_at_close_x.min():+.2f}")
print(f"  Max:    {y_diffs_at_close_x.max():+.2f}")

# If consistent, this is the Y offset to apply
if y_diffs_at_close_x.std() < 2.0:
    print(f"\n  Suggested Y_OFFSET for NPC: {-y_diffs_at_close_x.mean():.1f}")
